# MT Metadata Example 06

## Transfer Functions

Transfer functions are formated in various ways, this module attemps to make reading and writing between the various flavors easier.  Every format is read into a common container that can accommodate all metadata and statistical estimates.  This module makes no attempts to plot the data or analyze the data, that should be done using [MTpy v2.0](https://github.com/MTgeophysics/mtpy/tree/mpty_v2) (development to incorporate TF is in progress in version 2, version 1 mainly supports EDI files).  Here are some examples on how to use this module.

### Supported Formats

| Standard | Description | File Extension | Covariance | Reader | Writer |
|----------|-------------|----------------|------------|--------|--------|
| AVG | The output from Zonge's MTEdit processing code | .avg | No | Yes | No |
| EDI | Most common file type, "standard" is from the 1980's | .edi | No | Yes | Yes |
| EMTFXML | Format archived at IRIS, developed by Anna Kelbert | .xml | Yes | Yes | Yes |
| J-File | The output from Alan Chave's BIRRP processing code | .j | No | Yes | No |
| Z-file | The output from Gary Egbert's EMTF processing code | .zmm, .zrr, .zss | Yes | Yes | Yes |

All formats listed above are supported and have readers, some have writers as noted.  Each of these modules are in `mt_metadata.transfer_functions.io`.  A generic reader and writer is provided in the `mt_metadata.transfer_functions.core.TF` object described below.  Just call `TF.read_tf_file("path_to_file", **kwargs)` and the correct reader will be used.  

## TF Object

As noted above the `TF` object is a generic and common container for transfer function data and metadata.  The benefit is that from `TF` you can write your favorite file type, or as we will see store it in an MTH5 file which is agnostic to file type.  Moreover, the metadata in `TF` follows closely the same metadata standards for time series data, that way the only metadata needed to be added is that of how the transfer function was created.

In [1]:
from mt_metadata.transfer_functions.core import TF

## Structure of TF
The TF object stores the data as an xarray.DataSet, metadata are stored in a `mt_metadata.transfer_functions.tf.Station` and `mt_metadata.transfer_functions.tf.Survey` metadata containers to create a standard container. 

In [2]:
tf_object = TF()

### Survey Metadata
The container for survey metadata is `survey_metadata`, this includes `citation` information, `project` information, and general information about the survey the transfer function was collected for.  

In [3]:
print(tf_object.survey_metadata.to_json(required=False))

{
    "survey": {
        "acquired_by.author": null,
        "acquired_by.comments": null,
        "acquired_by.organization": null,
        "citation_dataset.authors": null,
        "citation_dataset.doi": null,
        "citation_dataset.journal": null,
        "citation_dataset.pages": null,
        "citation_dataset.title": null,
        "citation_dataset.volume": null,
        "citation_dataset.year": "1980-01-01T00:00:00+00:00",
        "citation_journal.authors": null,
        "citation_journal.doi": null,
        "citation_journal.journal": null,
        "citation_journal.pages": null,
        "citation_journal.title": null,
        "citation_journal.volume": null,
        "citation_journal.year": "1980-01-01T00:00:00+00:00",
        "comments": null,
        "country": null,
        "datum": "WGS84",
        "fdsn.alternate_code": null,
        "fdsn.alternate_network_code": null,
        "fdsn.channel_code": null,
        "fdsn.id": null,
        "fdsn.network": null,
       

### Station Metadata
The container for station metadata is `station_metadata` this includes important `location` information, `orientation`, `provenance`, and `transfer_function` information. This also includes `run` information and `channel` information.

In [4]:
print(tf_object.station_metadata.to_json(required=False))

{
    "station": {
        "acquired_by.author": null,
        "acquired_by.comments": null,
        "channel_layout": null,
        "channels_recorded": [
            "ex",
            "ey",
            "hx",
            "hy",
            "hz"
        ],
        "comments": null,
        "data_type": "BBMT",
        "doi": null,
        "fdsn.alternate_code": null,
        "fdsn.alternate_network_code": null,
        "fdsn.channel_code": null,
        "fdsn.id": null,
        "fdsn.network": null,
        "fdsn.new_epoch": null,
        "geographic_name": null,
        "id": "0",
        "location.country": null,
        "location.county": null,
        "location.datum": null,
        "location.declination.comments": null,
        "location.declination.epoch": null,
        "location.declination.model": "WMM",
        "location.declination.value": 0.0,
        "location.elevation": 0.0,
        "location.elevation_uncertainty": null,
        "location.latitude": 0.0,
        "location

## Data Container

The data container is an xarray.DataSet and convenience methods are included to get/set `impedance`, `tipper`, and statistical estimates of errors.  This includes covariance estimates like those output by EMTF. 

The `dataset` is setup with `input` and `output` coordinates, for the sake of generality the default for `input` and `output` channels are ex, ey, hx, hy, hz.  Any input/output combo that does not have a value is set to nan.

### Input Channels

These are source channels, for natural source MT this will be hx and hy

### Output Channels

These are the response channels, for natural source MT this will be ex, ey, and hz.

In [5]:
tf_object.dataset

<xarray.Dataset>
Dimensions:                        (period: 1, output: 5, input: 5)
Coordinates:
  * period                         (period) int64 1
  * output                         (output) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * input                          (input) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
Data variables:
    transfer_function              (period, output, input) complex128 (nan+na...
    transfer_function_error        (period, output, input) float64 nan ... nan
    transfer_function_model_error  (period, output, input) float64 nan ... nan
    inverse_signal_power           (period, output, input) complex128 (nan+na...
    residual_covariance            (period, output, input) complex128 0j ... 0j
Attributes: (12/14)
    survey:             0
    project:            None
    id:                 0
    name:               None
    latitude:           0.0
    longitude:          0.0
    ...                 ...
    datum:              None
    acquired_by:        None
    start:              1980-01-01T00:00:00+00:00
    end:                1980-01-01T00:00:00+00:00
    runs_processed:     ['0']
    coordinate_system:  geographic

The `dataset` also has attributes that are the important information to describe a transfer function and commonly used to make inversion files. These are pulled from `station_metadata` and `survey_metadata`. 

In [6]:
tf_object.station_metadata.id = "mt001"
tf_object.station_metadata.geographic_name = "Long descriptive name"
tf_object.station_metadata.location.latitude = "40:30:10.15"
tf_object.station_metadata.location.longitude = -120.7463
tf_object.station_metadata.location.elevation = 1123
tf_object.station_metadata.location.declination.value = -13.5
tf_object.station_metadata.location.datum = "WGS84"
tf_object.station_metadata.time_period.start = "2020-01-01T00:00:00"
tf_object.station_metadata.time_period.end = "2021-01-01T12:00:00"
tf_object.station_metadata.runs[0].id = "all"
tf_object.station_metadata.acquired_by.author = "MT Master"

tf_object.survey_metadata.project = "Test Project"
tf_object.survey_metadata.id = "CONUS"

tf_object.dataset

<xarray.Dataset>
Dimensions:                        (period: 1, output: 5, input: 5)
Coordinates:
  * period                         (period) int64 1
  * output                         (output) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * input                          (input) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
Data variables:
    transfer_function              (period, output, input) complex128 (nan+na...
    transfer_function_error        (period, output, input) float64 nan ... nan
    transfer_function_model_error  (period, output, input) float64 nan ... nan
    inverse_signal_power           (period, output, input) complex128 (nan+na...
    residual_covariance            (period, output, input) complex128 0j ... 0j
Attributes: (12/14)
    survey:             CONUS
    project:            Test Project
    id:                 mt001
    name:               Long descriptive name
    latitude:           40.50281944444444
    longitude:          -120.7463
    ...                 ...
    datum:              WGS84
    acquired_by:        MT Master
    start:              2020-01-01T00:00:00+00:00
    end:                2021-01-01T12:00:00+00:00
    runs_processed:     ['0']
    coordinate_system:  geographic

Use the convenience function `impedance`, `impedance_error`, `tipper`, `tipper_error` for accessing the common transfer function estimates.  There are also functions for `has_` which informs you if that estimate exists.

In [7]:
print("\n\t".join(["Attributes:"] + [func for func in dir(tf_object) if not callable(getattr(tf_object, func)) and not func.startswith("_")]))

Attributes:
	channel_nomenclature
	dataset
	elevation
	ex
	ex_ey
	ex_ey_hz
	ey
	fn
	frequency
	hx
	hx_hy
	hy
	hz
	impedance
	impedance_error
	impedance_model_error
	inverse_channel_nomenclature
	inverse_signal_power
	latitude
	logger
	longitude
	period
	residual_covariance
	run_metadata
	save_dir
	station
	station_metadata
	survey
	survey_metadata
	tf_id
	tipper
	tipper_error
	tipper_model_error
	transfer_function
	transfer_function_error
	transfer_function_model_error


In [8]:
print("\n\t".join(["Methods:"] + [func for func in dir(tf_object) if callable(getattr(tf_object, func)) and not func.startswith("_")]))

Methods:
	copy
	from_avg
	from_edi
	from_emtfxml
	from_jfile
	from_ts_station_metadata
	from_zmm
	from_zrr
	from_zss
	has_impedance
	has_inverse_signal_power
	has_residual_covariance
	has_tipper
	has_transfer_function
	make_zmm_run
	merge
	read
	read_tf_file
	to_avg
	to_edi
	to_emtfxml
	to_jfile
	to_ts_station_metadata
	to_zmm
	to_zrr
	to_zss
	write
	write_tf_file


#### Set period range

**Important**: set the periods before seting any statistical estimate. Otherwise you will get an error that the new estimate is not the same size as the old one and a new TF object should be initiated.

In [9]:
import numpy as np

n_periods = 6
tf_object.period = np.logspace(-3, 3, n_periods)

#### Set Impedance

**Note**: The dataset attributes are propogated through to each statistical estimate for easier book keeping.

In [10]:
tf_object.impedance = np.random.randn(n_periods, 2, 2) + np.random.randn(n_periods, 2, 2) * 1j

In [11]:
tf_object.has_impedance()

True

In [12]:
tf_object.impedance

<xarray.DataArray 'impedance' (period: 6, output: 2, input: 2)>
array([[[-1.31857217-1.82924421j,  0.48266317-2.24395282j],
        [-0.56334359-0.33198084j, -0.22136238+0.13775091j]],

       [[-0.68188511+0.35202552j,  0.23046569-1.07189555j],
        [ 0.87286675+1.5156209j , -1.06615417-0.53735351j]],

       [[-0.90563307+0.38815428j, -0.4336448 -0.69235157j],
        [-0.65110459-1.05047887j, -1.44697417+0.66651953j]],

       [[-0.71737506+1.3792712j , -0.76574522+0.55395412j],
        [ 0.09001671-1.15010703j, -0.07957938+0.54528415j]],

       [[ 0.95597527-0.48684218j, -0.40464395-1.05419323j],
        [-1.19144084+0.35921697j,  0.8432681 +1.19396708j]],

       [[ 0.74094607-0.90057442j, -0.91768692+0.7898233j ],
        [ 1.84222971-0.43048282j, -0.46752989+0.34032136j]]])
Coordinates:
  * period   (period) float64 0.001 0.01585 0.2512 3.981 63.1 1e+03
  * output   (output) <U2 'ex' 'ey'
  * input    (input) <U2 'hx' 'hy'
Attributes: (12/14)
    survey:             CONUS
    project:            Test Project
    id:                 mt001
    name:               Long descriptive name
    latitude:           40.50281944444444
    longitude:          -120.7463
    ...                 ...
    datum:              WGS84
    acquired_by:        MT Master
    start:              2020-01-01T00:00:00+00:00
    end:                2021-01-01T12:00:00+00:00
    runs_processed:     ['0']
    coordinate_system:  geographic

In [13]:
tf_object.dataset

<xarray.Dataset>
Dimensions:                        (period: 6, output: 5, input: 5)
Coordinates:
  * period                         (period) float64 0.001 0.01585 ... 63.1 1e+03
  * output                         (output) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
  * input                          (input) <U2 'ex' 'ey' 'hx' 'hy' 'hz'
Data variables:
    transfer_function              (period, output, input) complex128 (nan+na...
    transfer_function_error        (period, output, input) float64 nan ... nan
    transfer_function_model_error  (period, output, input) float64 nan ... nan
    inverse_signal_power           (period, output, input) complex128 (nan+na...
    residual_covariance            (period, output, input) complex128 0j ... 0j
Attributes: (12/14)
    survey:             CONUS
    project:            Test Project
    id:                 mt001
    name:               Long descriptive name
    latitude:           40.50281944444444
    longitude:          -120.7463
    ...                 ...
    datum:              WGS84
    acquired_by:        MT Master
    start:              2020-01-01T00:00:00+00:00
    end:                2021-01-01T12:00:00+00:00
    runs_processed:     ['0']
    coordinate_system:  geographic

### Get impedance element
We can use xarray type indexing to get at elements.  Here we are requesting the "Zyx" component and just the first element.

In [14]:
tf_object.impedance.loc[dict(input="hx", output="ey")][0]

<xarray.DataArray 'impedance' ()>
array(-0.56334359-0.33198084j)
Coordinates:
    period   float64 0.001
    output   <U2 'ey'
    input    <U2 'hx'
Attributes: (12/14)
    survey:             CONUS
    project:            Test Project
    id:                 mt001
    name:               Long descriptive name
    latitude:           40.50281944444444
    longitude:          -120.7463
    ...                 ...
    datum:              WGS84
    acquired_by:        MT Master
    start:              2020-01-01T00:00:00+00:00
    end:                2021-01-01T12:00:00+00:00
    runs_processed:     ['0']
    coordinate_system:  geographic

## Reading and Writing

Reading and writing are done through the methods `read_tf_file` and `write_tf_file`.  To write a file you must pass the new file name to be written.  If the extension is provided then the appropriate writer will be used, or you an spcify the writer using `file_type`.  

In [16]:
help(tf_object.write)

Help on method write in module mt_metadata.transfer_functions.core:

write(fn=None, save_dir=None, fn_basename=None, file_type='edi', **kwargs) method of mt_metadata.transfer_functions.core.TF instance
    Write an mt file, the supported file types are EDI and XML.
    
    .. todo:: j-files and avg files
    
    :param fn: full path to file to save to
    :type fn: :class:`pathlib.Path` or string
    
    :param save_dir: full path save directory
    :type save_dir: string
    
    :param fn_basename: name of file with or without extension
    :type fn_basename: string
    
    :param file_type: [ 'edi' | 'xml' | "zmm" ]
    :type file_type: string
    
    keyword arguments include
    
    :param longitude_format:  whether to write longitude as longitude or LONG.
                              options are 'longitude' or 'LONG', default 'longitude'
    :type longitude_format:  string
    :param latlon_format:  format of latitude and longitude in output edi,
                          